In [2]:
!git clone git@github.com:facebookresearch/ShapeR.git

Cloning into 'ShapeR'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 306 (delta 12), reused 6 (delta 5), pack-reused 280 (from 1)
Receiving objects: 100% (306/306), 41.25 MiB | 34.26 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Updating files: 100% (212/212), done.


In [ ]:
!pip install wheel setuptools ninja
!pip install numpy tqdm hydra-core matplotlib opencv-python imageio easydict munch plyfile
!pip install torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1 --index-url https://download.pytorch.org/whl/cu128
!pip install transformers trimesh scikit-image diffusers gradio peft einops
!pip install flash-attn --no-build-isolation --no-cache-dir
!pip install "imageio[ffmpeg]" "imageio[pyav]"
!pip install pymeshlab sophuspy fast_simplification scikit-learn timm plotly torchdiffeq sentencepiece protobuf pyrender jupyter

  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached plyfile-1.1.3-py3-none-any.whl.metadata (43 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)


In [1]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.7.1+cu128.html
!pip install --verbose git+https://github.com/nihalsid/torchsparse@legacy --no-build-isolation

Looking in links: https://data.pyg.org/whl/torch-2.7.1+cu128.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.2 MB/s eta 0:00:00ta 0:00:01
Using pip 25.0 from /opt/conda/lib/python3.12/site-packages/pip (python 3.12)
  Cloning https://github.com/nihalsid/torchsparse (to revision legacy) to /tmp/pip-req-build-gotuyqh4
  Running command git version
  git version 2.43.0
  Running command git clone --filter=blob:none https://github.com/nihalsid/torchsparse /tmp/pip-req-build-gotuyqh4
  Cloning into '/tmp/pip-req-build-gotuyqh4'...
  Running command git show-ref legacy
  20ccc92a3adceef1d88f63227b8e1d01b9c7ebcc refs/remotes/origin/legacy
  Running command git symbolic-ref -q HEAD
  refs/heads/master
  Running command git checkout -b legacy --track origin/legacy
  Switched to a new branch 'legacy'
  branch 'legacy' set up to track 'origin/legacy'.
  Resolved https://github.com/nihalsid/torchsparse to commit 20ccc92a3adceef1d88f63227b8e1d01b9c7ebcc
  Running command git rev-par

In [8]:
!pip install torch-sparse


  Using cached torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl


In [8]:
!pip install torch-scatter


  Using cached torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl


In [10]:
import torch
from torch_sparse import SparseTensor

device = "cuda" if torch.cuda.is_available() else "cpu"

row = torch.tensor([1, 4], device=device)
col = torch.tensor([2, 5], device=device)
value = torch.randn(2, device=device)

x = SparseTensor(row=row, col=col, value=value)

print("torch-sparse works ✅")
print(x)


torch-sparse works ✅
SparseTensor(row=tensor([1, 4], device='cuda:0'),
             col=tensor([2, 5], device='cuda:0'),
             val=tensor([-0.3385, -1.3784], device='cuda:0'),
             size=(5, 6), nnz=2, density=6.67%)


In [11]:
!mkdir -p data/14_secs/frames

In [14]:
!ls -lh data/14_secs.mp4

-rw-r--r-- 1 jovyan users 13M Feb  6 04:10 data/14_secs.mp4


In [6]:
import subprocess
import os

# First, try to repair/re-encode the video
print("Attempting to repair the video file...")
subprocess.run([
    'ffmpeg', '-i', 'data/14_secs.mp4',
    '-c', 'copy',
    '-movflags', 'faststart',
    'data/14_secs_fixed.mp4'
])

# Now extract frames from the fixed video
print("\nExtracting frames...")
os.makedirs('data/frames', exist_ok=True)
subprocess.run([
    'ffmpeg', '-i', 'data/14_secs_fixed.mp4',
    '-vf', 'fps=30',
    'data/frames/frame_%04d.png'
], check=True)

# Load frames
from PIL import Image
import numpy as np
import pickle
from pathlib import Path

frame_files = sorted(Path('data/frames').glob("*.png"))
print(f"Loading {len(frame_files)} frames...")

frames = [np.array(Image.open(f)) for f in frame_files]
frames_array = np.array(frames)

# Save as pickle
with open('data/frames.pkl', 'wb') as f:
    pickle.dump(frames_array, f)

print(f"\nDone!")
print(f"Frames shape: {frames_array.shape}")

Attempting to repair the video file...

Extracting frames...


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-2)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-alsa --enable-libpulse --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libmp3lame --enable-libaom --

CalledProcessError: Command '['ffmpeg', '-i', 'data/14_secs_fixed.mp4', '-vf', 'fps=30', 'data/frames/frame_%04d.png']' returned non-zero exit status 254.

In [1]:
!python video_to_frames.py

Extracting frames from data/output.mp4...
ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-2)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1743376049581/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-alsa --enable-libpulse --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx

In [1]:
import cv2
import numpy as np
import pickle

cap = cv2.VideoCapture('data/output.mp4')
frames = []

# Extract every Nth frame to reduce count
frame_skip = 3  # Only take every 3rd frame
frame_num = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_num % frame_skip == 0:
        # Resize to smaller resolution
        frame_resized = cv2.resize(frame, (720, 720))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)
    
    frame_num += 1

cap.release()

frames_array = np.array(frames)
with open('data/frames.pkl', 'wb') as f:
    pickle.dump(frames_array, f)

print(f"Done! Extracted {len(frames)} frames")
print(f"Shape: {frames_array.shape}")

Done! Extracted 105 frames
Shape: (105, 720, 720, 3)


In [1]:
!pip install omegaconf

  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)


In [2]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 189.1 kB/s eta 0:00:00a 0:00:01


In [3]:
# Install build dependencies
!pip install ninja
!apt-get update && apt-get install -y libsparsehash-dev

  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)


In [17]:
!apt-get update && apt-get install -y libsparsehash-dev

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)


In [18]:
# See where torchsparse is used in the codebase
!grep -r "torchsparse" ShapeR/ --include="*.py" | head -20

ShapeR/dataset/point_cloud.py:efficient 3D sparse convolutions via torchsparse.
ShapeR/dataset/point_cloud.py:import torchsparse
ShapeR/dataset/point_cloud.py:from torchsparse.utils.collate import sparse_collate
ShapeR/dataset/point_cloud.py:        sparse_tensor: torchsparse.SparseTensor.
ShapeR/dataset/point_cloud.py:        # Convert to torchsparse.SparseTensor
ShapeR/dataset/point_cloud.py:        pc_sparse_tensor = torchsparse.SparseTensor(
ShapeR/dataset/shaper_dataset.py:import torchsparse
ShapeR/dataset/shaper_dataset.py:            if isinstance(v, torchsparse.SparseTensor) or isinstance(v, torch.Tensor):
ShapeR/infer_shape.py:# important! We are using an old version of torchsparse, please use the legacy version otherwise you will get errors,\
ShapeR/infer_shape.py:# since torchsparse changed their datastructures in newer versions
ShapeR/model/flow_matching/shaper_denoiser.py:from torchsparse import SparseTensor
ShapeR/model/pointcloud_encoder.py:import torchsparse
ShapeR/mode

In [19]:
import os
import subprocess

# Create a local directory for sparsehash headers
!mkdir -p ~/local/include

# Download and extract sparsehash
!wget https://github.com/sparsehash/sparsehash/archive/refs/tags/sparsehash-2.0.4.tar.gz
!tar -xzf sparsehash-2.0.4.tar.gz
!cp -r sparsehash-sparsehash-2.0.4/src/sparsehash ~/local/include/
!cp -r sparsehash-sparsehash-2.0.4/src/google ~/local/include/

# Set environment variables to help the compiler find the headers
os.environ['CPLUS_INCLUDE_PATH'] = os.path.expanduser('~/local/include')
os.environ['C_INCLUDE_PATH'] = os.path.expanduser('~/local/include')

# Now try installing torchsparse
!pip install git+https://github.com/mit-han-lab/torchsparse.git

--2026-02-07 04:59:55--  https://github.com/sparsehash/sparsehash/archive/refs/tags/sparsehash-2.0.4.tar.gz
140.82.114.4thub.com (github.com)... 
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/sparsehash/sparsehash/tar.gz/refs/tags/sparsehash-2.0.4 [following]
--2026-02-07 04:59:56--  https://codeload.github.com/sparsehash/sparsehash/tar.gz/refs/tags/sparsehash-2.0.4
140.82.112.10eload.github.com (codeload.github.com)... 
connected. to codeload.github.com (codeload.github.com)|140.82.112.10|:443... 
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘sparsehash-2.0.4.tar.gz’

sparsehash-2.0.4.ta     [ <=>                ] 315.58K  --.-KB/s    in 0.1s    

2026-02-07 04:59:56 (2.41 MB/s) - ‘sparsehash-2.0.4.tar.gz’ saved [323154]

  Cloning https://github.com/mit-han-lab/torchsparse.git to /tmp/pip-req-build-ducmirjw
  Running com

In [3]:
# We need to generate the sparseconfig.h file
!cd sparsehash-sparsehash-2.0.4 && ./configure --prefix=$HOME/local
!cd sparsehash-sparsehash-2.0.4 && make install

# Now try installing torchsparse again
import os
os.environ['CPLUS_INCLUDE_PATH'] = os.path.expanduser('~/local/include')
os.environ['C_INCLUDE_PATH'] = os.path.expanduser('~/local/include')

!pip install git+https://github.com/mit-han-lab/torchsparse.git

checking for a BSD-compatible install... /usr/bin/install -c
yescking whether build environment is sane... 
checking for a thread-safe mkdir -p... /usr/bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking for g++... g++
checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
noecking whether we are cross compiling... 
ohecking for suffix of object files... 
checking whether we are using the GNU C++ compiler... yes
yescking whether g++ accepts -g... 
checking for style of include used by make... GNU
gcc3king dependency style of g++... 
checking for gcc... gcc
yescking whether we are using the GNU C compiler... 
yescking whether gcc accepts -g... 
none neededr gcc option to accept ISO C89... 
gcc3king dependency style of gcc... 
gcc -Eng how to run the C preprocessor... 
checking for grep that handles long lines and -e... /usr/bin/grep
che

In [21]:
!python -c "$(curl -fsSL https://raw.githubusercontent.com/mit-han-lab/torchsparse/master/install.py)"


Traceback (most recent call last):
  File "<string>", line 40, in <module>
KeyError: 'torch25'


In [4]:
!pip install einops

  Using cached einops-0.8.2-py3-none-any.whl.metadata (13 kB)
Using cached einops-0.8.2-py3-none-any.whl (65 kB)


In [5]:
!pip install sophuspy

  Using cached sophuspy-1.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.8 kB)
Using cached sophuspy-1.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (238 kB)


In [6]:
!pip install torchdiffeq

  Using cached torchdiffeq-0.2.5-py3-none-any.whl.metadata (440 bytes)
Using cached torchdiffeq-0.2.5-py3-none-any.whl (32 kB)


In [7]:
!pip install torch_cluster

  Using cached torch_cluster-1.6.3-cp312-cp312-linux_x86_64.whl


In [33]:
!python -c "import pickle; f = open('ShapeR/dataset/shaper_dataset.py', 'r'); [print(f'{i}: {line}', end='') for i, line in enumerate(f.readlines()[50:70], 50)]; f.close()"

50:     def __getitem__(self, idx):
51:         sample_path = self.paths[idx]
52:         pkl_sample = pickle.load(open(sample_path, "rb"))
53: 
54:         semi_dense_points = pkl_sample["points_model"].numpy()
55:         bounds = pkl_sample["bounds"].numpy()
56:         semi_dense_theta = pkl_sample["inv_dist_std"].numpy()[:]
57:         semi_dense_phi = pkl_sample["dist_std"].numpy()[:]
58:         scale = 0.9 / np.max(bounds)
59:         semi_dense_points[:, :3] *= scale
60: 
61:         valid = np.all(np.abs(semi_dense_points) <= 1.0, axis=-1)
62:         semi_dense_points = semi_dense_points[valid, :]
63:         semi_dense_theta = semi_dense_theta[valid]
64:         semi_dense_phi = semi_dense_phi[valid]
65: 
66:         # threshold the semi-dense points
67:         selected_semi_dense_points = semi_dense_points
68: 
69:         thres_theta = self.config.dataset.semi_dense_threshold_theta


In [34]:
!python -c "import pickle; sample_keys = ['points_model', 'bounds', 'inv_dist_std', 'dist_std', 'images', 'masks', 'poses']; print('Expected pickle keys:', sample_keys)"

Expected pickle keys: ['points_model', 'bounds', 'inv_dist_std', 'dist_std', 'images', 'masks', 'poses']


In [35]:
!ls ShapeR/preprocessing/

__init__.py  helper.py	     projection_utils.py  view_selection_heuristic.py
__pycache__  point_cloud.py  ray_utils.py
camera.py    pose.py	     tensor_wrapper.py


In [37]:
!cat ShapeR/README.md

## ShapeR: Robust Conditional 3D Shape Generation from Casual Captures

![teaser](resources/Teaser.jpg)

ShapeR introduces a novel approach to metric shape generation. Given an input image sequence, preprocessing extracts per-object metric sparse SLAM points, images, poses, and captions using off-the-shelf methods. A rectified flow transformer operating on VecSet latents conditions on these multimodal inputs to generate a shape code, which is decoded into the object’s mesh. By applying the model object-centrically to each detected object, we obtain a metric reconstruction of the entire scene.

[Project Page](http://facebookresearch.github.io/ShapeR)  |  [Paper](https://cdn.jsdelivr.net/gh/facebookresearch/ShapeR@main/resources/ShapeR.pdf) | [Arxiv](https://arxiv.org/abs/2601.11514)  |  [Video](https://www.youtube.com/watch?v=EbY30KAA55I)  |  [HF-Model](https://huggingface.co/facebook/ShapeR/)  |  [HF Evaluation Dataset](https://huggingface.co/datasets/facebook/ShapeR-Evaluation)

## In

In [31]:
!python ShapeR/infer_shape.py --input_pkl data/frames.pkl --config balance --output_dir output --is_local_path

Fetching 5 files: 100%|████████████████████████| 5/5 [00:00<00:00, 99864.38it/s]
Loading model...
/home/jovyan/ShapeR/model/dinov2/layers/attention.py:34: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/jovyan/ShapeR/model/dinov2/layers/block.py:42: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
/home/jovyan/ShapeR/model/dinov2/layers/swiglu_ffn.py:55: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/jovyan/ShapeR/model/dinov2/hub/backbones.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the